## そもそもARモデルをしっかり学習できるのかを確かめる

次時刻予測をさせる

In [1]:
import argparse
import os
path = os.getcwd()
path=path[:path.find('timeseries-WGAN')+15]
import numpy as np
from scipy import stats
import math
import sys
sys.path.append(path+"/")
import random
import statsmodels.api as sm
from scipy.stats import norm

import torchvision.transforms as transforms
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable

In [2]:
# 人工データを生成してくれる機械が置いてあるところ
import tsModel
# 学習用のニューラルネットが置いてあるところ
import models

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
# フォルダを作成（既にあるならそれで良し）
os.makedirs("output-images", exist_ok=True)
os.makedirs("parameters", exist_ok=True)

In [5]:
p=7

## NNで確かめる

In [6]:
i=0

In [8]:
net = models.LinearPredictNet(p = p, input_dim=1, is_bias=True)

In [10]:
parameter_path = "change_seed/parameters/network_epoch20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed{0}.pth".format(i)
parameter_path

'change_seed/parameters/network_epoch20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth'

In [11]:
net.load_state_dict(torch.load(parameter_path))

<All keys matched successfully>

In [12]:
net.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.3131, -0.3650,  0.2718, -0.4386,  0.3368, -0.4201,  0.2601, -0.0106]])),
             ('fc1.bias', tensor([0.0559]))])

In [20]:
net.state_dict()["fc1.weight"][0]

tensor([ 0.3131, -0.3650,  0.2718, -0.4386,  0.3368, -0.4201,  0.2601, -0.0106])

In [24]:
net.state_dict()["fc1.bias"].item()

0.055861249566078186

In [39]:
params = []
mu = []

for i in range(10):
    net = models.LinearPredictNet(p = p, input_dim=1, is_bias=True)
    parameter_path = "change_seed/parameters/network_epoch20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed{0}.pth".format(i)
    net.load_state_dict(torch.load(parameter_path))
    params.append(np.array(net.state_dict()["fc1.weight"][0]))
    mu.append(net.state_dict()["fc1.bias"].item())

In [30]:
params

[array([ 0.31314477, -0.36504686,  0.27184433, -0.43856618,  0.33677167,
        -0.42010248,  0.26005104, -0.01064355], dtype=float32),
 array([ 0.29983357, -0.3969987 ,  0.27157873, -0.38687065,  0.2769836 ,
        -0.3992239 ,  0.2647001 ,  0.01933478], dtype=float32),
 array([ 0.27080438, -0.39342478,  0.28358912, -0.40527612,  0.22151943,
        -0.41049236,  0.2654277 ,  0.0069396 ], dtype=float32),
 array([ 0.29360873, -0.44982007,  0.2733666 , -0.43622828,  0.27736217,
        -0.42202893,  0.27404442, -0.00865356], dtype=float32),
 array([ 0.2563577 , -0.38253444,  0.32640874, -0.47440857,  0.26235667,
        -0.3748935 ,  0.30355993, -0.00252023], dtype=float32),
 array([ 0.32526284, -0.35008043,  0.2832891 , -0.35309443,  0.29167408,
        -0.37545577,  0.29418257, -0.01228052], dtype=float32),
 array([ 0.23107277, -0.40981576,  0.25780845, -0.38802522,  0.26731107,
        -0.37267247,  0.26718327,  0.00619552], dtype=float32),
 array([ 0.32252413, -0.40755153,  0.3034

In [40]:
mu

[0.055861249566078186,
 0.0794120505452156,
 -0.09066326916217804,
 -0.09570492804050446,
 0.016815723851323128,
 0.0342823751270771,
 0.022684520110487938,
 0.02085406892001629,
 0.07357200980186462,
 -0.03404462710022926]

In [36]:
print(np.array(params).mean(axis = 0))
print(np.array(params).std(axis = 0))

[ 0.28889686 -0.39398235  0.28126055 -0.40571442  0.2783966  -0.3943136
  0.28805017  0.00151257]
[0.02822203 0.02613973 0.01899416 0.03525268 0.02718427 0.01918498
 0.02692717 0.01054511]


In [41]:
print(np.array(mu).mean())
print(np.array(mu).std())

0.008306917361915111
0.05924134751805972


## 最小二乗法で確かめる 最尤推定と同じ？

In [ ]:
ar_data = np.array(trainData.view(-1))
ar_data.shape

In [ ]:
model = sm.tsa.AR(ar_data)

In [ ]:
print(model.select_order(maxlag=10, ic='aic'))

In [ ]:
result = model.fit(maxlag=p)

In [ ]:
print(result.params)

In [ ]:
print(result.sigma2)

In [ ]:
Data = tsModel.SARIMA(a=[0.3,-0.4,0.3,-0.4,0.3,-0.4,0.3], N=1400, random_seed=0, sigma=2)
Data = torch.tensor(Data, dtype=torch.float)
Data = torch.tensor(Data)
Data=Data.view(1,-1)
trainData = Data[:,:1000]

In [ ]:
ar_data = np.array(trainData.view(-1))
model = sm.tsa.AR(ar_data)
result = model.fit(maxlag=p)
print(result.params)
print(result.sigma2)

In [ ]:
params = []
sigma2 = []
for i in range(10):
    Data = tsModel.SARIMA(a=[0.3,-0.4,0.3,-0.4,0.3,-0.4,0.3], N=1400, random_seed=i, sigma=2)
    Data = torch.tensor(Data, dtype=torch.float)
    Data = torch.tensor(Data)
    Data=Data.view(1,-1)
    trainData = Data[:,:1000]

    ar_data = np.array(trainData.view(-1))
    model = sm.tsa.AR(ar_data)
    result = model.fit(maxlag=p)
    params.append(result.params)
    sigma2.append(result.sigma2)

In [ ]:
np.array(params).mean(axis=0)

In [ ]:
np.array(params).std(axis=0)

In [ ]:
np.array(sigma2).mean(axis=0)

In [ ]:
np.sqrt(sigma2).mean()

In [ ]:
np.sqrt(sigma2).std()